In [1]:
using Revise

In [2]:
using MeshCat
using MeshCatMechanisms
using RigidBodyDynamics
using Gurobi
using LCPSim

In [3]:
vis = Visualizer()
IJuliaCell(vis)

Listening on 127.0.0.1:7005...
zmq_url=tcp://127.0.0.1:6005
web_url=http://127.0.0.1:7005/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x0000000005bffc40, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000005c063c0, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000054c9fc0, 68, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000005c063c0, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000054c9fc0, 68, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7005/static/", "tcp://127.0.0.1:6005", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict{String,MeshCat.SceneTrees.SceneNode}()), 7005, HttpServer.Server(HttpServer.HttpHandler(MeshCat.ZMQServer.handle_file_request, Base.TCPServer(RawFD(49) active), Dict{String,Function}(Pair{String,Function}("error", HttpServer.#1),Pair{String,Function}("listen", HttpServer.#3))), WebSockets.WebSocketHandler(MeshCat.ZMQServer.#5)), ZMQ.Context(Ptr{Void} @0x00000000054c1080, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000005c0b290, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005a7e780, 56, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000005b0e110, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000054cc520, 57, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x00000000058c8bd0, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005bcc2c0, 58, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000005bda890, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000054a1a60, 59, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000005bcee70, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005a680b0, 60, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000005bcc430, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005bc9e40, 61, (1, 0), Condition(Any[Task (runnable) @0x00007fb387d9df90]), (true, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000005bcc430, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005bc9e40, 61, (1, 0), Condition(Any[Task (runnable) @0x00007fb387d9df90]), (true, false), 0)), "tcp://127.0.0.1:6005")), false)

In [30]:
urdf = "box_atlas.urdf"
mechanism = parse_urdf(Float64, urdf)
floating_base = findjoint(mechanism, "floating_base")
floating_base.position_bounds .= RigidBodyDynamics.Bounds(-10, 10)
floating_base.velocity_bounds .= RigidBodyDynamics.Bounds(-1000, 1000)
floating_base.effort_bounds .= RigidBodyDynamics.Bounds(0, 0)
env = LCPSim.parse_contacts(mechanism, urdf, 1.0, :xz)
rh = findbody(mechanism, "rh")
lh = findbody(mechanism, "lh")
rf = findbody(mechanism, "rf")
lf = findbody(mechanism, "lf")
floor = findbody(mechanism, "floor")
wall = findbody(mechanism, "wall")
LCPSim.filter_contacts!(env, mechanism, Dict([rh => [], lh => [wall], rf => [floor], lf => [floor, wall]]))
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf), vis);

In [31]:
xstar = MechanismState{Float64}(mechanism)
set_configuration!(xstar, findjoint(mechanism, "floating_base"), [0, 1.05, 0])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_extension"), [0.8])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_extension"), [0.8])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_extension"), [0.5])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_extension"), [0.5])
set_configuration!(mvis, configuration(xstar))

In [42]:
x0 = MechanismState(mechanism)
set_configuration!(x0, configuration(xstar))
set_velocity!(x0, randn(num_velocities(x0)) - 1)
results = LCPSim.simulate(x0, x -> zeros(num_velocities(x)), env, 0.05, 30, GurobiSolver(Gurobi.Env(), OutputFlag=0));

Academic license - for non-commercial use only


In [43]:
for r in results
    set_configuration!(mvis, configuration(r.state))
    sleep(0.05)
end